In [1]:
from  urllib  import request
import logging
from pathlib import Path
import numpy as np
import pandas as pd
import re
import MeCab
from gensim import corpora, models,matutils
import random
from tqdm import tqdm_notebook as tqdm
from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report

/Users/tdual/anaconda2/envs/py3.6/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/tdual/anaconda2/envs/py3.6/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
mecab = MeCab.Tagger("-Ochasen -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd/")

In [3]:
res = request.urlopen("http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt")
stopwords = [line.decode("utf-8").strip() for line in res]
print(stopwords[:3])

['あそこ', 'あたり', 'あちら']


In [4]:
res = request.urlopen("http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/English.txt")
stopwords += [line.decode("utf-8").strip() for line in res]
print(stopwords[-3:])

["you've", 'z', 'zero']


In [5]:
class Tokenizer:
    def __init__(self, stopwords, parser=None, include_pos=None, exclude_posdetail=None, exclude_reg=None):
    
        self.stopwords = stopwords
        self.include_pos = include_pos if include_pos else  ["名詞", "動詞", "形容詞"]
        self.exclude_posdetail = exclude_posdetail if exclude_posdetail else ["接尾", "数"]
        self.exclude_reg = exclude_reg if exclude_reg else r"$^"  # no matching reg
        if parser:
            self.parser = parser
        else:
            mecab = MeCab.Tagger("-Ochasen -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd/")
            self.parser = mecab.parse
            

    def tokenize(self, text, show_pos=False):
        text = re.sub(r"https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+", "", text)    #URL
        text = re.sub(r"\"?([-a-zA-Z0-9.`?{}]+\.jp)\"?" ,"", text)  # xxx.jp 
        text = text.lower()
        l = [line.split("\t") for line in self.parser(text).split("\n")]
        res = [
            i[2] if not show_pos else (i[2],i[3]) for i in l 
                if len(i) >=4 # has POS.
                    and i[3].split("-")[0] in self.include_pos
                    and i[3].split("-")[1] not in self.exclude_posdetail
                    and not re.search(r"(-|−)\d", i[2])
                    and not re.search(self.exclude_reg, i[2])
                    and i[2] not in self.stopwords          
            ]
        return res

In [6]:
t = Tokenizer(stopwords, mecab.parse, exclude_reg=r"\d(年|月|日)")

In [7]:
t.tokenize("認めたくないものだな。自分自身の若さ故の過ちというものを。")

['認める', '自分自身', '若さ故の過ち']

In [8]:
pos_doc = []
neg_doc = []

In [9]:
with open("data/amazon_ja/pos.txt") as f:
    pos_doc = [t.tokenize(doc) for doc in tqdm(f.readlines())]
print(pos_doc[:5])

HBox(children=(IntProgress(value=0, max=31206), HTML(value='')))


[['目当て', '映画', 'やる', 'おる', 'たまたまこ', '作品', '観る', 'なる', '申し訳ない', 'ディズニー映画', '比べる', '地味', 'キャラクター', '感じる', '期待', 'する', 'いる', 'の', '観る', 'いる', '素晴らしい', '映像', '音楽', '魅力', 'キャラクター', '世界観', '引き込む', 'いく', 'ラスト', '近づく', '意外', '展開', '家族愛', '夫婦。', '耐える', 'きれる', '涙', '涙', '周り', '子供達', '親', 'すすり泣く', '声', '映画館', '響く', 'いる', '家族', '友人', '大切さ', '考える', '映画', '家族', '友人', '感謝', 'する', '気持ち', '大切', 'する', '気持ち', 'なる', '娘。', '小さい', 'の', '大きい', 'なる', '見せる', 'Amazon', '発見', 'する', '予約', 'する', '娘', '一緒', '観る', 'の', '楽しみ', '映画館', '見れる', '良い'], ['映画', '見る', 'の', '偶然', 'GW', '最終日', '映画', '見る', '映画館', 'つく', 'の', 'リメンバー・ミー', '泣ける', '広告', '噂', '聞く', 'いる', '疑心暗鬼', '心', 'ある', '号泣', 'する', 'いる', '死者', '美しい', '背景', 'キャラクター', '繊細', '動き', '表情', '最高', '歌声', '愛情', '深い', '内容', 'やる', 'Pixar', 'アニメ', '見る', 'いる', '泣く', 'の', '初め', '涙', 'あふれる', 'くる', 'の', 'わかる', '涙', 'とまる', '後日', '上映期間', '迫る', 'いる', 'の', '見る', 'なる', '鑑賞', '上映', '終了', '間際', 'お客', '一人', '号泣', '是非', 'リメンバー・ミー', '見る', '言葉', '意味', '忘れないで', '意識', 'する', 'みる', 'ほしい', '感じる'

In [10]:
with open("data/amazon_ja/neg.txt") as f:
    neg_doc = [t.tokenize(doc) for doc in tqdm(f.readlines())]
print(neg_doc[:5])

HBox(children=(IntProgress(value=0, max=4002), HTML(value='')))


[['他人', 'ブック・オブ・ライフ', '観る', 'てる', 'ん', '良い', 'ん'], ['個人', 'MovieNEX', '最大', '旨味', 'Digital Copy', '不便', 'なる', 'しまう', 'MCU', '作品', 'ディズニー', '権限', '作品', 'プライム', 'ビデオ', 'プライム', '会員限定', '追加', 'する', '外出', '潰す', 'Digital Copy', '大変', '便利', 'niconicodrm', 'プラットフォーム', '見る', '発売', '作品', '一方', 'プラットフォーム', 'Google Play', 'デノミ', '視聴', 'なる', 'しまう', 'Google', '吹替え', '視聴', 'niconico', '吹き替え', '字幕', '視聴', '可能', 'コード', '1回', '使用', 'niconico', '登録', 'する', 'コード', 'Google', '使える', 'の', 'niconico', 'Google', '2つ', 'アプリ', '使い分ける', 'ブラックパンサー', '吹き替え', '視聴', 'なる', '細かい', 'しれる', 'めんどくさい', 'なる', '残念', '予想', 'する', 'さら', '残念', 'MCU', '次作', 'インフィニティ・ウォー', 'Digital Copy', '吹き替え', '視聴', 'なる', '考える', '字幕', '見る', '済む', '便利', '非常', '非常', '残念', '改善', 'する', 'いただく'], ['劇場', '感じる', '退屈', '飽きる', '映画', 'コミック', '原作', 'ある', '逸脱', '出来る', 'しょうが', 'ハーブ', 'ドラッグ', '飲む', '超人', 'する', '説得力', '車', '乗る', '屋根', '乗る', 'の', 'デフォ', '笑える', '要素', '有る', '全体', 'すごい', 'の', '神秘', 'せい', 'やっつける', 'プロット', 'すぎる'], ['MarBell', 'シリーズ', '面白い', '見る', 'ア

In [11]:
d = corpora.Dictionary(pos_doc+neg_doc)

In [12]:
pos_bow = [d.doc2bow(doc) for doc  in tqdm(pos_doc)]

HBox(children=(IntProgress(value=0, max=31206), HTML(value='')))

In [13]:
neg_bow = [d.doc2bow(doc) for doc  in tqdm(neg_doc)]

HBox(children=(IntProgress(value=0, max=4002), HTML(value='')))

In [17]:
df = pd.DataFrame([len(b) for b in pos_bow+neg_bow],columns=["length"])

In [18]:
df.head()

,length
0,60
1,73
2,76
3,46
4,94


In [19]:
df.describe()

,length
count,35208.000000
mean,22.224523
std,21.811964
min,1.000000
25%,11.000000
50%,17.000000
75%,27.000000
max,803.000000


In [20]:
len(d)

38308

In [21]:
dense = list(matutils.corpus2dense(pos_bow+neg_bow,  num_terms=len(d)))

In [22]:
dense = np.array(dense)

In [23]:
dense.shape

(38308, 35208)

In [24]:
pos_label = [1 for b in pos_doc]
neg_label = [0 for b in neg_doc]
print(len(pos_label))
print(len(neg_label))

31206
4002


In [25]:
label = pos_label + neg_label

In [26]:
len(label)

35208

In [27]:
data_train_s, data_test_s, label_train_s, label_test_s = model_selection.train_test_split(dense.T, label, test_size=0.1)

In [28]:
estimator = RandomForestClassifier(verbose=True)

In [29]:
estimator.fit(data_train_s, label_train_s)

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:  3.8min finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=True,
            warm_start=False)

In [30]:
estimator.score(data_test_s, label_test_s)

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.1s finished


0.9113888099971599

In [31]:
tuned_parameters = [{'n_estimators': [50, 70, 90, 110]}]#, 130, 150]}]#, 'max_features': ['auto', 'sqrt', 'log2', None]}]

clf = GridSearchCV(RandomForestClassifier(), tuned_parameters, cv=2, scoring='accuracy', n_jobs=-1)

In [ ]:
clf.fit(data_train_s, label_train_s)

In [ ]:
print("==== グリッドサーチ")
print("  ベストパラメタ")
print(clf.best_estimator_)

In [ ]:
print("トレーニングデータでCVした時の平均スコア")
for params, mean_score, all_scores in clf.grid_scores_:
        print("{:.3f} (+/- {:.3f}) for {}".format(mean_score, all_scores.std() / 2, params))

In [ ]:
y_true, y_pred = label_test_s, clf.predict(data_test_s)
print(classification_report(y_true, y_pred,target_names=["nag","pos"]))